# Training Pipeline

We choose LSTM as a classification model 


### Imports & Load data

In [56]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
import torch 
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

In [6]:
# parameters
max_len = 60 

voc_size = 5000

### Connect to Hopsworks

In [17]:
import hopsworks

project = hopsworks.login() 

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5322


Connected. Call `.close()` to terminate connection gracefully.


In [19]:
try: 
    feature_view = fs.get_feature_view(
    name = 'headlines_sentiment_fg',
    version = 2)
except:
    feature_group= fs.get_feature_group(name = 'headlines_sentiment_fg', version=2)
    query = feature_group.select_all()
    feature_view = fs.create_feature_view(name = 'headlines_sentiment_fg', 
                                        version=2,
                                        labels=["sentiment"],
                                        query=query)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/5322/fs/5242/fv/headlines_sentiment_fg/version/2


In [112]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.2)


2023-01-08 21:06:09,738 INFO: USE `evaengel_featurestore`
2023-01-08 21:06:10,599 INFO: SELECT `fg0`.`sentiment` `sentiment`, `fg0`.`headline` `headline`
FROM `evaengel_featurestore`.`headlines_sentiment_fg_2` `fg0`


In [114]:
X_train['headline'] = X_train['headline'].apply(lambda x: np.fromstring(x.replace('[','').replace(']',''), dtype=int, sep=',').tolist())
X_test['headline'] = X_test['headline'].apply(lambda x: np.fromstring(x.replace('[','').replace(']',''), dtype=int, sep=',').tolist())
X_train = X_train['headline'].to_list()
X_test = X_test['headline'].to_list()

## LSTM Model

In [120]:
embedding_vector_features=40

model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length = max_len))
model.add(Dropout(0.5))
model.add(LSTM(200))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [121]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
141/141 [==============================] - 28s 166ms/step - loss: 0.3806 - accuracy: 0.3449 - val_loss: 0.2719 - val_accuracy: 0.3478
Epoch 2/10
141/141 [==============================] - 22s 158ms/step - loss: 0.3482 - accuracy: 0.3445 - val_loss: 0.2796 - val_accuracy: 0.3478
Epoch 3/10
141/141 [==============================] - 21s 148ms/step - loss: 0.3517 - accuracy: 0.3445 - val_loss: 0.2901 - val_accuracy: 0.3478
Epoch 4/10
141/141 [==============================] - 24s 170ms/step - loss: 0.3483 - accuracy: 0.3445 - val_loss: 0.2724 - val_accuracy: 0.3478
Epoch 5/10
141/141 [==============================] - 20s 141ms/step - loss: 0.3399 - accuracy: 0.3445 - val_loss: 0.2727 - val_accuracy: 0.3478
Epoch 6/10
141/141 [==============================] - 20s 139ms/step - loss: 0.3525 - accuracy: 0.3445 - val_loss: 0.2801 - val_accuracy: 0.3478
Epoch 7/10
141/141 [==============================] - 19s 134ms/step - loss: 0.3470 - accuracy: 0.3445 - val_loss: 0.2787 - val_ac